Link: https://www.kaggle.com/code/toufikalhajj/bitcoin-prediction-2025-2026-time-series-analysis

In [ ]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType
import pandas as pd
import pyspark.pandas as ps
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import boxcox
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm
import numpy as np

plt.style.use('seaborn-v0_8')


spark = SparkSession.builder.appName("Bitcoin").getOrCreate()

Data preparation

In [ ]:
data_path = "/Users/pepijnschouten/Desktop/Python_Scripts/" \
      "Python_scripts_Varia/Timeseries/crypto_tsa/" \
            "sarimax_btc_example/data/btcusd_1-min_data.csv"
    
df = (spark
      .read
      .csv(data_path, header=True, inferSchema=True)
      .withColumnRenamed("Timestamp", "Unix Timestamp")
)

df.printSchema()

In [ ]:
df = df.withColumn("Timestamp",
                   F.from_unixtime("Unix Timestamp", "yyyy-MM-dd HH:mm:ss"))

df.show()

In [ ]:
# calculate mean price
df = df.withColumn("Avg Price",
                   F.expr("+".join(['Open', 'High', 'Low', 'Close'])) / 4
                   )

df.show()

df.select("Avg Price").summary().show()
df.summary("count").show()

In [ ]:
# filter null values
df = df.filter(F.col("Unix Timestamp").isNotNull())

In [ ]:
# Plot the data
ps.DataFrame(df).plot(x="Timestamp", y="Avg Price")

In [ ]:
# convert to pandas
df_pandas = df.select("Unix Timestamp", "Avg Price").toPandas()

df_pandas['Timestamp'] = pd.to_datetime(df_pandas['Unix Timestamp'], unit="s")
df_pandas = df_pandas.set_index('Timestamp').drop('Unix Timestamp', axis=1)

In [ ]:
df_truncate = df_pandas.truncate(before=datetime(2017, 1, 1, 0, 0, 0),
                                #   after=datetime(2022, 1, 1, 0, 0, 0)
)

# df_truncate = df_pandas

print(df_truncate.head())

In [ ]:
# resample the data
df_D = df_truncate.resample('D').mean()
df_M = df_truncate.resample('ME').mean()
df_Q = df_truncate.resample('QE').mean()
df_Y = df_truncate.resample('YE').mean()

df_dict = {"Days": df_D, "Months": df_M, "Quarters": df_Q, "Years": df_Y}

fig, axs = plt.subplots(2, 2,
                       figsize=(20, 10),
                       tight_layout=True,
                       sharey=True,)

for (k, v), ax in zip(df_dict.items(), axs.flat):
    v.plot(ax=ax)
    ax.set_title(k)
    ax.grid(True, color='k', linewidth=0.5)

plt.show()



Autoregressive model (AR)

Check stationarity of the time series (Dickey-Fuller, KPSS)

In [ ]:
# augmented dickey-fuller test (below 0.05)
print("Dickey-Fuller test")
for k, v in df_dict.items():
    p_value = adfuller(v, regression="ct")[1]
    p_bool = p_value <= 0.05
    print(f"{k:10} | {p_value:.4f} {"<" if p_bool else ">"} 0.05 | Stationary: {p_value <= 0.05}")

# Kwiatkowski–Phillips–Schmidt–Shin (KPSS) tests (above 0.05)
print("KPSS test")
for k, v in df_dict.items():
    p_value = kpss(v)[1]
    p_bool = p_value >= 0.05
    print(f"{k:10} | {p_value:.4f} {">" if p_bool else "<"} 0.05 | Stationary: {p_value >= 0.05}")

Seasonal decomposition of raw data

In [ ]:
# seasional decomposition
for k, v in df_dict.items():
    result = seasonal_decompose(v, model='multiplicative')
    
    _, axs = plt.subplots(4, 1, sharex=True)
    result.observed.plot(ax=axs[0])
    axs[0].set_title('Observed')
    
    result.trend.plot(ax=axs[1])
    axs[1].set_title('Trend')
    
    result.seasonal.plot(ax=axs[2])
    axs[2].set_title('Seasonal')
    
    result.resid.plot(ax=axs[3])
    axs[3].set_title('Residuals')
    
    for ax in axs:
        ax.set_ylabel("Price")
    
    plt.suptitle(k)
    plt.show()

In [ ]:
# drop years
df_dict = {"Days": df_D, "Months": df_M, "Quarters": df_Q}

Box-Cox Transformation

In [ ]:
# boxcox
for v in df_dict.values():
    v['boxcox'] = boxcox(v['Avg Price'])[0]
    
# augmented dickey-fuller test (below 0.05)
print("Dickey-Fuller test")
for k, v in df_dict.items():
    p_value = adfuller(v['boxcox'], regression="ct")[1]
    p_bool = p_value <= 0.05
    print(f"{k:10} | {p_value:.4f} {"<" if p_bool else ">"} 0.05 | Stationary: {p_value <= 0.05}")

In [ ]:
# boxcox with shift
shift = 1
for v in df_dict.values():
    v['boxcox_diff'] = v['boxcox'] - v['boxcox'].shift(shift)
    
# augmented dickey-fuller test (below 0.05)
print("Dickey-Fuller test")
for k, v in df_dict.items():
    p_value = adfuller(v['boxcox_diff'][shift:], regression="ct")[1]
    p_bool = p_value <= 0.05
    print(f"{k:10} | {p_value:.4f} {"<" if p_bool else ">"} 0.05 | Stationary: {p_value <= 0.05}")

In [ ]:
seasonal_decompose(df_M['boxcox_diff'][shift:], model='additive').plot()
plt.show()

ACF Plot (AutoCorrelationFunction)

In [ ]:
fig, ax = plt.subplots(2, 1, tight_layout=True)
sm.graphics.tsa.plot_acf(df_M['boxcox_diff'][shift:].values.squeeze(), lags=15, ax=ax[0])
sm.graphics.tsa.plot_pacf(df_M['boxcox_diff'][shift:].values.squeeze(), lags=15, ax=ax[1])
plt.show()

Fitting the SARIMAX model (to-do)

In [ ]:
import warnings
import statsmodels.api as sm
from itertools import product

# Set up parameter ranges for the SARIMAX grid search
Qs = range(0, 2)  # Seasonal AR order
qs = range(0, 3)  # Non-seasonal AR order
Ps = range(0, 3)  # Seasonal MA order
ps = range(0, 3)  # Non-seasonal MA order

# Seasonal difference (D) and non-seasonal difference (d) values
D = 1
d = 1

# Generate all combinations of parameters for SARIMAX
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)

# Initialize variables to store the best model and results
best_aic = float("inf")
best_model = None
best_param = None
results = []

# Suppress warnings during the search to keep the output clean
warnings.filterwarnings('ignore')

# Grid search for optimal SARIMAX parameters
for param in parameters_list:
    try:
        # Fit the SARIMAX model with current parameter combination
        model = sm.tsa.statespace.SARIMAX(df_M['boxcox'], 
                                          order=(param[0], d, param[1]), 
                                          seasonal_order=(param[2], D, param[3], 12)).fit(disp=-1)
        
        # Extract the AIC of the current model
        aic = model.aic
        
        # If the current model has a lower AIC, update the best model
        if aic < best_aic:
            best_aic = aic
            best_model = model
            best_param = param
        
        
        results.append([param, aic])
    
    except ValueError as e:
     
        print(f"Error with parameters {param}: {e}")
        continue

# Display the best parameter combination and corresponding AIC
print(f"Best AIC: {best_aic} with parameters: {best_param}")

Inverse box-cox transformation function

In [ ]:
def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda * y + 1) / lmbda))

Future prediction

In [ ]:
df_M_pred = df_M.copy()

start_date = df_M.index[-1] + pd.Timedelta(days=40)
end_date = start_date + pd.Timedelta(weeks=1000)

print(start_date, end_date)

date_list = pd.date_range(start=start_date, end=end_date, freq='M')

future = pd.DataFrame(index=date_list, columns=df_M.columns)
df_M_pred = pd.concat([df_M_pred, future])

# make predictions
predictions = best_model.predict(start=0,
                                 end=len(df_M_pred) - 1)

print(predictions)

df_M_pred['predictions'] = invboxcox(predictions,
                                     lmbda=boxcox(df_M["Avg Price"])[1])

# plot results
plt.figure()
df_M_pred[['Avg Price', 'predictions']].plot()